# Answer questions from Reddit SEC filing with Opper and Mistral

In [3]:
# Install the datasets library from huggingface
!pip install opperai -U
!pip install pydantic


  Attempting uninstall: opperai
    Found existing installation: opperai 0.5.1
    Uninstalling opperai-0.5.1:
      Successfully uninstalled opperai-0.5.1


## Imports

In [4]:
import os
from opperai import fn, Client, AsyncClient
from opperai.types.indexes import RetrievalResponse
from pydantic import BaseModel, Field
from typing import List, Literal

os.environ["OPPER_API_KEY"] = "op-BN3K7QS92CIJI16MANUE"

opper = Client()


None


## Index PDF

In [73]:

# We get or create our index
index = opper.indexes.get(name="mistral-rag2")
if not index:
    index = opper.indexes.create(name="mistral-rag2")

    print(index)

    # we upload our pdf to the index
    opper.indexes.upload_file(
        id=index.id, 
        file_path="./reddit-sec.pdf",
        )


id=1717 name='mistral-rag2' created_at=datetime.datetime(2024, 5, 7, 6, 42, 14, 348702, tzinfo=TzInfo(UTC))


{'uuid': 'b32a719b-bccb-4503-9cb8-71f6462bca7d',
 'key': '34/1717/b32a719b-bccb-4503-9cb8-71f6462bca7d',
 'original_filename': 'reddit-sec.pdf',
 'document_id': 14950}

## Retrieve relevant content for answering question

In [105]:
question = "What are the key financial and growth numbers for Reddit?"

# We retrieve the content from the index
results = opper.indexes.retrieve(
    id=index.id,
    query=question,
    k=3
)

print(results)


AttributeError: 'int' object has no attribute 'id'

## Create response with citations

In [ ]:
class Citation(BaseModel):
    source: str = Field(..., description="The source, such as file name, used to answer the question")
    page_number: int = Field(..., description="The page number of the source")
    citation: str = Field(..., description="A relevant citation from the source, preferably short")

class Response(BaseModel):
    answer: str = Field(..., description="The answer to the question")
    citations: List[Citation] = Field(..., description="Citations used to answer the question")

@fn(path="test/mistral-rag/response", model="azure/mistral-large-eu")
def answer_question(question: str, sources: List[RetrievalResponse]) -> Response:
    """ Produce an answer to the question using the sources. Clearly mark each fact or statement with inline citations in the style of [1], [2]. 
    """

response = answer_question(question, sources)
print(response)


answer="Reddit's key financial and growth numbers include a Daily Active Unique (DAUq) growth of 27% compared to the prior year period and 11% compared to the prior quarter period. The Weekly Active Unique (WAUq) also showed significant growth, with a 29% increase compared to the prior year period, and 12% compared to the prior quarter period [1]. In terms of revenue, Reddit recorded revenue of $804.0 million for the year ended December 31, 2023, a 21% increase compared to the prior year period [2]. The digital advertising market, excluding China and Russia, is expected to grow at a CAGR of 20% to $1.0 trillion in 2027, and the user economy on Reddit is estimated to grow at a CAGR of 12% to $2.1 trillion in 2027 [3]. Additionally, there was approximately 35% growth in the number of videos watched for 10 seconds or more and an approximately 16% increase in daily active video viewers compared to December 2022 [4]. Reddit also observed an increase of over 30% in 'Good Visits,' defined as 

## Print it! 

In [ ]:
print(response.answer)
print()
index = 1
for citation in response.citations:
    print(f"[{index}]", citation.source, "page:", citation.page_number, "segment:", f'"{citation.citation[:50]}..."')
    index += 1
    


Reddit's key financial and growth numbers include a Daily Active Unique (DAUq) growth of 27% compared to the prior year period and 11% compared to the prior quarter period. The Weekly Active Unique (WAUq) also showed significant growth, with a 29% increase compared to the prior year period, and 12% compared to the prior quarter period [1]. In terms of revenue, Reddit recorded revenue of $804.0 million for the year ended December 31, 2023, a 21% increase compared to the prior year period [2]. The digital advertising market, excluding China and Russia, is expected to grow at a CAGR of 20% to $1.0 trillion in 2027, and the user economy on Reddit is estimated to grow at a CAGR of 12% to $2.1 trillion in 2027 [3]. Additionally, there was approximately 35% growth in the number of videos watched for 10 seconds or more and an approximately 16% increase in daily active video viewers compared to December 2022 [4]. Reddit also observed an increase of over 30% in 'Good Visits,' defined as a user c